In [1]:
import logging
import time
import lotr

logging.basicConfig(level=logging.INFO, format='%(asctime)s: %(message)s')

In [2]:
conf = lotr.read_conf()
sets = lotr.get_sets(conf)

2020-09-07 21:48:45,934: Reading project configuration...
2020-09-07 21:48:45,934: ...Reading project configuration (0.0s)
2020-09-07 21:48:45,950: Getting all sets to work on...
2020-09-07 21:48:47,726: ...Getting all sets to work on (1.776s)


In [3]:
timestamp = time.time()
lotr.clear_project_folders()
lotr.download_sheet(conf)

changes = False
for set_data in sets:
    set_id, set_name, set_row = set_data
    if 'octgn' in conf['outputs']:
        lotr.generate_octgn_xml(conf, set_id, set_name, set_row)

    for lang in conf['languages']:
        lotr.generate_xml(conf, set_id, set_name, set_row, lang)
        lotr.update_xml(conf, set_id, set_name, lang)
        new_hash, old_hash = lotr.calculate_hashes(set_id, set_name, lang)
        if new_hash != old_hash:
            changes = True

        lotr.copy_raw_images(conf, set_id, set_name, lang)
        lotr.copy_xml(set_id, set_name, lang)

if changes:
    lotr.create_project()
else:
    logging.info('No changes since the last run, skipping creating the'
                 ' project')

logging.info('Done (%ss)', round(time.time() - timestamp, 3))

2020-09-07 21:48:55,665: Clearing the project folders...
2020-09-07 21:48:55,719: ...Clearing the project folders (0.053s)
2020-09-07 21:48:55,734: Downloading cards spreadsheet from Google Drive...
2020-09-07 21:48:58,912: ...Downloading cards spreadsheet from Google Drive (3.178s)
2020-09-07 21:48:58,912: [ALeP - Children of Eorl, English] Backing up a previous Strange Eons xml file...
2020-09-07 21:48:58,912: [ALeP - Children of Eorl, English] ...Backing up a previous Strange Eons xml file (0.0s)
2020-09-07 21:48:58,912: [ALeP - Children of Eorl, English] Generating xml file for Strange Eons...
2020-09-07 21:49:01,480: [ALeP - Children of Eorl, English] ...Generating xml file for Strange Eons (2.568s)
2020-09-07 21:49:01,480: [ALeP - Children of Eorl, English] Updating the Strange Eons xml file with additional data...
2020-09-07 21:49:01,518: [ALeP - Children of Eorl, English] ...Updating the Strange Eons xml file with additional data (0.038s)
2020-09-07 21:49:01,534: [ALeP - Childr

### Strange Eons Part:

1. Open `setGenerator.seproject` in Strange Eons.
2. Run `Script/makeCards` script by double clicking it.
3. Once completed, close Strange Eons (wait until it finished packing the project).

In [4]:
timestamp = time.time()
for set_data in sets:
    set_id, set_name, _ = set_data
    for lang in conf['languages']:
        skip_set, skip_ids = lotr.get_skip_info(set_id, set_name, lang)
        if skip_set:
            logging.info('[%s, %s] No changes since the last run,'
                         ' skipping', set_name, lang)
            continue

        if 'db_octgn' in conf['outputs']:
            lotr.generate_jpg300_nobleed(set_id, set_name, lang, skip_ids)
            if 'db' in conf['outputs']:
                lotr.generate_db(set_id, set_name, lang)
            if 'octgn' in conf['outputs']:
                lotr.generate_octgn(set_id, set_name, lang)

        if 'pdf' in conf['outputs']:
            lotr.generate_png300_pdf(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_pdf(set_id, set_name, lang)

        if 'makeplayingcards' in conf['outputs']:
            lotr.generate_png800_bleedmpc(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_mpc(conf, set_id, set_name, lang)

        if 'drivethrucards' in conf['outputs']:
            lotr.generate_jpg300_bleeddtc(conf, set_id, set_name, lang, skip_ids)
            lotr.generate_dtc(conf, set_id, set_name, lang)

if 'octgn' in conf['outputs'] and conf['octgn_destination_path']:
    lotr.copy_octgn_outputs(conf)

logging.info('Done (%ss)', round(time.time() - timestamp, 3))

2020-09-07 21:59:22,914: [ALeP - Children of Eorl, English] Getting skip information...
2020-09-07 21:59:22,930: [ALeP - Children of Eorl, English] ...Getting skip information (0.016s)
2020-09-07 21:59:22,930: [ALeP - Children of Eorl, English] Generating images for PDF outputs...
2020-09-07 21:59:50,365: [ALeP - Children of Eorl, English] CompletedProcess(args='"C:\\Program Files\\GIMP 2\\bin\\gimp-console-2.10.exe" -i -b "(python-prepare-pdf-back-folder 1 \\"Temp\\\\generate_png300_pdf.8a3273ca-1ccd-4e07-913b-766fcc49fe6f.English\\" \\"imagesEons\\\\png300PDF\\\\8a3273ca-1ccd-4e07-913b-766fcc49fe6f.English\\")" -b "(gimp-quit 0)"', returncode=0, stdout=b'EEEEeEeek! 45 GeglBuffers leaked\n', stderr=b'batch command executed successfully\nC:\\Program Files\\GIMP 2\\bin\\gimp-console-2.10.exe: GEGL-WARNING: (../gegl/buffer/gegl-tile-handler-cache.c:1076):gegl_tile_cache_destroy: runtime check failed: (g_queue_is_empty (&cache_queue))\n')
2020-09-07 22:04:09,079: [ALeP - Children of Eorl,

PermissionError: [Errno 13] Permission denied: 'Output\\PDF\\ALeP - Children of Eorl.English\\A4.ALeP - Children of Eorl.English.pdf'

Now there should be the following outputs:

- `Output/OCTGN/<set name>/`: `<octgn id>/set.xml` and `<set name>.<language>.o8c` image packs for OCTGN (300 dpi JPG).
- `Output/DB/<set name>.<language>/`: 300 dpi JPG images for general purposes.
- `Output/PDF/<set name>.<language>/`: PDF files in `A4` and `letter` format for home printing.
- `Output/DriveThruCards/<set name>.<language>/`: `zip` and `7z` archives of 300 dpi PNG images to be printed on DriveThruCards.com.
- `Output/MakePlayingCards/<set name>.<language>/`: `zip` and `7z` archives of 800 dpi PNG images to be printed on MakePlayingCards.com.

Additionally, if you specified `octgn_destination_path`, all OCTGN outputs are packed into a destination archive.